<a href="https://colab.research.google.com/github/prasanth9989/assignment/blob/main/pspyk12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
#user_data
%%writefile user_data.csv
user_id,series,season,timestamp,genre,duration_mins
521,”Mirzapur”,3,2024-07-30 15:00:00,action,300
672,”Panchayat”,3,2024-07-30 15:00:00,comedy,200
197,”Family Man”,2,2024-07-30 15:00:00,action,500
521,”Mirzapur”,2,2024-07-29 15:00:00,action,280
211,”Queens Gambit”,1,2024-07-30 15:00:00,drama,170
521,”Mirzapur”,1,2024-07-28 15:00:00,action,230
844,”Westworld”,3,2024-07-30 15:00:00,sci-fi,310
672,”Panchayat”,3,2024-07-29 15:00:00,comedy,210
256,”Homecoming”,2,2024-07-30 15:00:00,thriller,310
489,”Outer Range”,1,2024-07-30 15:00:00,sci-fi,340
200,”Black Mirror”,2,2024-07-30 15:00:00,sci-fi,140
256,”Outer Range”,2,2024-07-30 15:00:00,thriller,250
489,”Outer Range”,2,2024-07-28 15:00:00,sci-fi,170
200,”Black Mirror”,3,2024-07-29 15:00:00,sci-fi,190
672,”Panchayat”,2,2024-07-28 15:00:00,comedy,160
672,”Outer Range”,1,2024-07-25 15:00:00,sci-fi,250
200,”Black Mirror”,4,2024-07-28 15:00:00,sci-fi,200
844,”Westworld”,2,2024-07-29 15:00:00,sci-fi,300
672,”Black Mirror”,5,2024-07-28 15:00:00,sci-fi,150
672,”Panchayat”,1,2024-07-27 15:00:00,comedy,190

Overwriting user_data.csv


In [44]:
#Import Libraries
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [45]:
#Create SparkSession for app (Streaming Analysis)
spark=SparkSession.builder.appName('streaming Analysis').getOrCreate()

In [47]:
#Create Dataframe series_df
df=spark.read.option('header','True').csv('user_data.csv')
df.show()

+-------+---------------+------+-------------------+--------+-------------+
|user_id|         series|season|          timestamp|   genre|duration_mins|
+-------+---------------+------+-------------------+--------+-------------+
|    521|     ”Mirzapur”|     3|2024-07-30 15:00:00|  action|          300|
|    672|    ”Panchayat”|     3|2024-07-30 15:00:00|  comedy|          200|
|    197|   ”Family Man”|     2|2024-07-30 15:00:00|  action|          500|
|    521|     ”Mirzapur”|     2|2024-07-29 15:00:00|  action|          280|
|    211|”Queens Gambit”|     1|2024-07-30 15:00:00|   drama|          170|
|    521|     ”Mirzapur”|     1|2024-07-28 15:00:00|  action|          230|
|    844|    ”Westworld”|     3|2024-07-30 15:00:00|  sci-fi|          310|
|    672|    ”Panchayat”|     3|2024-07-29 15:00:00|  comedy|          210|
|    256|   ”Homecoming”|     2|2024-07-30 15:00:00|thriller|          310|
|    489|  ”Outer Range”|     1|2024-07-30 15:00:00|  sci-fi|          340|
|    200| ”B

In [48]:
a=df.select('series').distinct().show()

+---------------+
|         series|
+---------------+
|   ”Family Man”|
|   ”Homecoming”|
|    ”Panchayat”|
|  ”Outer Range”|
| ”Black Mirror”|
|     ”Mirzapur”|
|    ”Westworld”|
|”Queens Gambit”|
+---------------+



In [49]:
#Find the user with maximum watchtime
arr1=df.groupBy('user_id').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending=False).show(1)

+-------+------------------+
|user_id|sum(duration_mins)|
+-------+------------------+
|    672|            1160.0|
+-------+------------------+
only showing top 1 row



In [50]:
#Calculate overall total Watchtime
arr2=df.groupBy('user_id').agg({'duration_mins':'sum'}).show()

+-------+------------------+
|user_id|sum(duration_mins)|
+-------+------------------+
|    521|             810.0|
|    200|             530.0|
|    672|            1160.0|
|    256|             560.0|
|    197|             500.0|
|    211|             170.0|
|    844|             610.0|
|    489|             510.0|
+-------+------------------+



In [51]:
#Find most popular shows (based on watchtime)
arr3=df.groupBy('series','genre').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending=False).show()

+---------------+--------+------------------+
|         series|   genre|sum(duration_mins)|
+---------------+--------+------------------+
|     ”Mirzapur”|  action|             810.0|
|  ”Outer Range”|  sci-fi|             760.0|
|    ”Panchayat”|  comedy|             760.0|
| ”Black Mirror”|  sci-fi|             680.0|
|    ”Westworld”|  sci-fi|             610.0|
|   ”Family Man”|  action|             500.0|
|   ”Homecoming”|thriller|             310.0|
|  ”Outer Range”|thriller|             250.0|
|”Queens Gambit”|   drama|             170.0|
+---------------+--------+------------------+



In [52]:
#Find most popular shows (based on user popularity)
arr4=df.groupBy('series').agg({'user_id':'count'}).orderBy('count(user_id)',ascending=False).show()

+---------------+--------------+
|         series|count(user_id)|
+---------------+--------------+
|    ”Panchayat”|             4|
|  ”Outer Range”|             4|
| ”Black Mirror”|             4|
|     ”Mirzapur”|             3|
|    ”Westworld”|             2|
|   ”Family Man”|             1|
|   ”Homecoming”|             1|
|”Queens Gambit”|             1|
+---------------+--------------+



In [53]:
#Find the most popular genre
arr5=df.groupBy('genre').agg({'genre':'count'}).orderBy('count(genre)',ascending=False).show(1)

+------+------------+
| genre|count(genre)|
+------+------------+
|sci-fi|           9|
+------+------------+
only showing top 1 row



In [54]:
#Calculate total watchtime per user
arr6=df.groupBy('user_id').agg({'duration_mins':'sum'}).show()

+-------+------------------+
|user_id|sum(duration_mins)|
+-------+------------------+
|    521|             810.0|
|    200|             530.0|
|    672|            1160.0|
|    256|             560.0|
|    197|             500.0|
|    211|             170.0|
|    844|             610.0|
|    489|             510.0|
+-------+------------------+



In [55]:
df.show()

+-------+---------------+------+-------------------+--------+-------------+
|user_id|         series|season|          timestamp|   genre|duration_mins|
+-------+---------------+------+-------------------+--------+-------------+
|    521|     ”Mirzapur”|     3|2024-07-30 15:00:00|  action|          300|
|    672|    ”Panchayat”|     3|2024-07-30 15:00:00|  comedy|          200|
|    197|   ”Family Man”|     2|2024-07-30 15:00:00|  action|          500|
|    521|     ”Mirzapur”|     2|2024-07-29 15:00:00|  action|          280|
|    211|”Queens Gambit”|     1|2024-07-30 15:00:00|   drama|          170|
|    521|     ”Mirzapur”|     1|2024-07-28 15:00:00|  action|          230|
|    844|    ”Westworld”|     3|2024-07-30 15:00:00|  sci-fi|          310|
|    672|    ”Panchayat”|     3|2024-07-29 15:00:00|  comedy|          210|
|    256|   ”Homecoming”|     2|2024-07-30 15:00:00|thriller|          310|
|    489|  ”Outer Range”|     1|2024-07-30 15:00:00|  sci-fi|          340|
|    200| ”B

In [56]:
#Find most popular genre (based on engagement count)
arr7=df.groupBy('genre').agg({'duration_mins':'count'}).show()

+--------+--------------------+
|   genre|count(duration_mins)|
+--------+--------------------+
|  action|                   4|
|   drama|                   1|
|thriller|                   2|
|  sci-fi|                   9|
|  comedy|                   4|
+--------+--------------------+



In [57]:
arr8=df.groupBy('genre').agg({'genre':'count'}).show()

+--------+------------+
|   genre|count(genre)|
+--------+------------+
|  action|           4|
|   drama|           1|
|thriller|           2|
|  sci-fi|           9|
|  comedy|           4|
+--------+------------+



In [58]:
#Find average watchtime per genre
arr9=df.groupBy('genre').agg({'duration_mins':'avg'}).show()

+--------+------------------+
|   genre|avg(duration_mins)|
+--------+------------------+
|  action|             327.5|
|   drama|             170.0|
|thriller|             280.0|
|  sci-fi|227.77777777777777|
|  comedy|             190.0|
+--------+------------------+



In [59]:
df.show()

+-------+---------------+------+-------------------+--------+-------------+
|user_id|         series|season|          timestamp|   genre|duration_mins|
+-------+---------------+------+-------------------+--------+-------------+
|    521|     ”Mirzapur”|     3|2024-07-30 15:00:00|  action|          300|
|    672|    ”Panchayat”|     3|2024-07-30 15:00:00|  comedy|          200|
|    197|   ”Family Man”|     2|2024-07-30 15:00:00|  action|          500|
|    521|     ”Mirzapur”|     2|2024-07-29 15:00:00|  action|          280|
|    211|”Queens Gambit”|     1|2024-07-30 15:00:00|   drama|          170|
|    521|     ”Mirzapur”|     1|2024-07-28 15:00:00|  action|          230|
|    844|    ”Westworld”|     3|2024-07-30 15:00:00|  sci-fi|          310|
|    672|    ”Panchayat”|     3|2024-07-29 15:00:00|  comedy|          210|
|    256|   ”Homecoming”|     2|2024-07-30 15:00:00|thriller|          310|
|    489|  ”Outer Range”|     1|2024-07-30 15:00:00|  sci-fi|          340|
|    200| ”B

In [70]:
#Find peak traffic days
#(Output 1 = Full Date)
arr10=df.groupBy(to_date('timestamp')).agg(count('timestamp')).orderBy('count(timestamp)',ascending=False)
arr10.select('to_date(timestamp)').show(1)


+------------------+
|to_date(timestamp)|
+------------------+
|        2024-07-30|
+------------------+
only showing top 1 row



In [71]:
#(Output 2 = Only Day)
arr10=df.groupBy(dayofmonth('timestamp')).agg(count('timestamp')).orderBy('count(timestamp)',ascending=False)
arr10.select('dayofmonth(timestamp)').show(1)


+---------------------+
|dayofmonth(timestamp)|
+---------------------+
|                   30|
+---------------------+
only showing top 1 row



In [72]:
df.show()

+-------+---------------+------+-------------------+--------+-------------+
|user_id|         series|season|          timestamp|   genre|duration_mins|
+-------+---------------+------+-------------------+--------+-------------+
|    521|     ”Mirzapur”|     3|2024-07-30 15:00:00|  action|          300|
|    672|    ”Panchayat”|     3|2024-07-30 15:00:00|  comedy|          200|
|    197|   ”Family Man”|     2|2024-07-30 15:00:00|  action|          500|
|    521|     ”Mirzapur”|     2|2024-07-29 15:00:00|  action|          280|
|    211|”Queens Gambit”|     1|2024-07-30 15:00:00|   drama|          170|
|    521|     ”Mirzapur”|     1|2024-07-28 15:00:00|  action|          230|
|    844|    ”Westworld”|     3|2024-07-30 15:00:00|  sci-fi|          310|
|    672|    ”Panchayat”|     3|2024-07-29 15:00:00|  comedy|          210|
|    256|   ”Homecoming”|     2|2024-07-30 15:00:00|thriller|          310|
|    489|  ”Outer Range”|     1|2024-07-30 15:00:00|  sci-fi|          340|
|    200| ”B

In [ ]:
#Find the user with most diverse show preference



In [ ]:
#Find the binge-watchers


In [60]:
#Find the user with longest watching streak
arr11=

+-------+------------------+
|user_id|sum(duration_mins)|
+-------+------------------+
|    672|            1160.0|
|    521|             810.0|
|    844|             610.0|
|    256|             560.0|
|    200|             530.0|
|    489|             510.0|
|    197|             500.0|
|    211|             170.0|
+-------+------------------+



In [81]:
#Total Seasons available
arr12=df.groupBy('series').agg(max('season')).orderBy('max(season)',ascending=False).show()

+---------------+-----------+
|         series|max(season)|
+---------------+-----------+
| ”Black Mirror”|          5|
|     ”Mirzapur”|          3|
|    ”Panchayat”|          3|
|    ”Westworld”|          3|
|   ”Family Man”|          2|
|   ”Homecoming”|          2|
|  ”Outer Range”|          2|
|”Queens Gambit”|          1|
+---------------+-----------+



In [74]:
#Fetch a list of all series
arr13=df.select('series').distinct()
arr13.rdd.flatMap(lambda x:[x[0]]).collect()

['”Family Man”',
 '”Homecoming”',
 '”Panchayat”',
 '”Outer Range”',
 '”Black Mirror”',
 '”Mirzapur”',
 '”Westworld”',
 '”Queens Gambit”']